In [1]:
import numpy as np
import itertools
import pandas as pd
from sklearn.datasets import load_svmlight_file
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
from twelm import TWELM, XELM, RBFNet



from sklearn.metrics import confusion_matrix


In [ ]:
%matplotlib tk
%load_ext autoreload
%autoreload 0

In [2]:
datafiles = [
    r'data/5ht2a_ExtFP.libsvm',
    r'data/5ht2c_ExtFP.libsvm',
    r'data/5ht6_ExtFP.libsvm',
    r'data/5ht7_ExtFP.libsvm',
    r'data/M1_ExtFP.libsvm',
    r'data/SERT_ExtFP.libsvm',
    r'data/cathepsin_ExtFP.libsvm',
    r'data/d2_ExtFP.libsvm',
    r'data/h1_ExtFP.libsvm',
    r'data/hERG_ExtFP.libsvm',
    r'data/hiv_integrase_ExtFP.libsvm',
    r'data/hiv_protease_ExtFP.libsvm',
             ]

In [3]:
def process_cm(confusion_mat, i=0):
    # i means which class to choose to do one-vs-the-rest calculation
    # rows are actual obs whereas columns are predictions
    TP = confusion_mat[i,i]  # correctly labeled as i
    FP = confusion_mat[:,i].sum() - TP  # incorrectly labeled as i
    FN = confusion_mat[i,:].sum() - TP  # incorrectly labeled as non-i
    TN = confusion_mat.sum().sum() - TP - FP - FN
    return TP, FP, FN, TN

In [4]:
def bac_error(Y, Y_predict):
    cm = confusion_matrix(Y, Y_predict)
    bac_values = np.zeros(cm.shape[0])
    for i in xrange(cm.shape[0]):
        tp, fp, fn, tn = process_cm(cm, i=i)
        if tp+fn > 0 and tn+fp>0:
            bac_values[i] = 0.5*tp/(tp+fn) + 0.5*tn/(tn+fp)
    return bac_values

In [5]:
def bac_scorer(estimator, X, Y):
    Y_predict = estimator.predict(X)
    bac_values = bac_error(Y, Y_predict)
    return np.mean(bac_values)

In [6]:
def perform_grid_search(estimator, features, activity,scorer,param_grid, n_outer_folds, n_inner_folds, n_outer_repetitions):
    """
    returns
    test score for each outer fold
    best score from grid search
    best estimator parameters for each iteration
    """
    test_scores = np.zeros(n_outer_folds*n_outer_repetitions)
    train_scores = np.zeros(n_outer_folds*n_outer_repetitions)
    best_parameters = []
    for rep in range(n_outer_repetitions):
        #print('%d/%d' % (rep, n_outer_repetitions))
        fold = StratifiedKFold(activity, n_folds=n_outer_folds, shuffle=True)
        #print(len(fold))
        fit_params = {}
        if isinstance(estimator, XELM) and 'h' in param_grid:
            max_h = min(features.shape[0], max(param_grid['h']))
#             fit_params['hidden_layer'] = features[np.random.choice(features.shape[0],
#                                             max_h,
#                                             replace=False)]
        for i, (train_index, test_index) in enumerate(fold):
            search = GridSearchCV(estimator,\
                                  param_grid,\
                                  scoring=scorer,
                                  cv=n_inner_folds,
                                  n_jobs=1)

            search.fit(features[train_index], activity[train_index])

            test_score = scorer(search.best_estimator_, features[test_index], activity[test_index])
            test_scores[rep*n_outer_folds+i] = test_score
            train_scores[rep*n_outer_folds+i] = search.best_score_

            best_parameters.append(search.best_params_)
#             print('train score=%f, test score=%f' % (search.best_score_, test_score))
            print(search.best_params_)
    return test_scores, train_scores, best_parameters

In [7]:
def test_models(estimators, estimator_grids, X, Y, scorer, n_outer_folds, n_inner_folds, n_outer_repetitions):
    estimator_scores = np.zeros(len(estimators))
    estimator_scores_std = np.zeros(len(estimators))
    assert len(estimators) == len(estimator_grids)
    for i, (estimator, grid) in enumerate(itertools.izip(estimators, estimator_grids)):
        print(type(estimator).__name__)
        
        scores_test, _, _ = perform_grid_search(estimator,
                                                X,
                                                Y,
                                                param_grid=grid,
                                                scorer=scorer,
                                                n_outer_folds=n_outer_folds,
                                                n_inner_folds=n_inner_folds, 
                                                n_outer_repetitions=n_outer_repetitions)
        print(scores_test)
        estimator_scores[i]= scores_test.mean()
        estimator_scores_std[i] = scores_test.std()
    
    return estimator_scores, estimator_scores_std
        

In [10]:
estimators = [
    RBFNet(h=100, f='tanimoto'),
    XELM(h=10, f='tanimoto', balanced=True),
    RBFNet(h=100, f='kulczynski2'),
    XELM(h=10, f='kulczynski2', balanced=True),
    RandomForestClassifier(n_jobs=-1)
]

estimator_grids = [
    #{'C': [ 1000,10000, 100000], 'h': [ 1500, 2000, 2500]},
    {'C': [ 1000, 100000], 'h': [ 500,  1500], 'b':[0.4, 1, 2]},
    {'C': [ 1000, 100000], 'h': [ 500,  1500]},
    {'C': [ 1000, 100000], 'h': [ 500,  1500], 'b':[0.4, 1, 2]},
    {'C': [ 1000, 100000], 'h': [ 500,  1500]},
    {'n_estimators': [75, 125]}
]

estimator_grids_simple = [
    #{'C': [ 1000,10000, 100000], 'h': [ 1500, 2000, 2500]},
    {'C': [ 1000], 'h': [   1500], 'b':[0.4,  2]},
    {'C': [ 1000], 'h': [   1500]},
    {'C': [ 1000], 'h': [   1500], 'b':[0.4,  2]},
    {'C': [ 1000], 'h': [ 1500]},
    {'n_estimators': [75, 125]}
]



In [11]:
def get_estimator_descritpion(estimator):
    name = type(estimator).__name__
    if hasattr(estimator, 'metric_name'):
        name+= '(%s)' % estimator.metric_name
    return name

scores_grid = pd.DataFrame()
scores_grid.loc[:, 'dataset'] = pd.Series()
for estimator in estimators:
    scores_grid.loc[:,get_estimator_descritpion(estimator) +'_score'] = pd.Series(np.zeros(len(datafiles)))
    scores_grid.loc[:, get_estimator_descritpion(estimator)+'_std'] = pd.Series(np.zeros(len(datafiles)))

for i, datafile in enumerate(datafiles):
    print(datafile)
    
    scores_grid.ix[i, 'dataset'] = datafile
    features, activity = load_svmlight_file(datafile)

    
    estimators_scores, estimatoirs_scores_std = test_models(estimators, 
                                                            estimator_grids_simple,
                                                            features,
                                                            activity,
                                                            scorer=bac_scorer,
                                                            n_outer_folds=4,
                                                            n_inner_folds=4,
                                                            n_outer_repetitions=2)
    
    
    for estimator, score, std in itertools.izip(estimators, estimators_scores, estimatoirs_scores_std):
        scores_grid.ix[i, get_estimator_descritpion(estimator)+'_score'] = score
        scores_grid.ix[i, get_estimator_descritpion(estimator)+'_std'] = std
    
    

data/5ht2a_ExtFP.libsvm
RBFNet
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 0.4}
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 0.4}
{'h': 1500, 'C': 1000, 'b': 0.4}
[ 0.8428304   0.84559207  0.8564086   0.8783678   0.87661481  0.85892479
  0.8668876   0.87561795]
XELM
{'h': 1500, 'C': 1000}
{'h': 1500, 'C': 1000}
{'h': 1500, 'C': 1000}
{'h': 1500, 'C': 1000}
{'h': 1500, 'C': 1000}
{'h': 1500, 'C': 1000}
{'h': 1500, 'C': 1000}
{'h': 1500, 'C': 1000}
[ 0.86604376  0.87049311  0.84994937  0.85639985  0.87182792  0.86906625
  0.86035165  0.86910892]
RBFNet
{'h': 1500, 'C': 1000, 'b': 0.4}
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 0.4}
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 0.4}
{'h': 1500, 'C': 1000, 'b': 0.4}
{'h': 1500, 'C': 1000, 'b': 2}
{'h': 1500, 'C': 1000, 'b': 2}
[ 0.8756022   0.87501918  0.84626715  0.86998435  0.86010617  0.8

KeyboardInterrupt: 

In [ ]:
import cPickle
n = RBFNet(h=100)
with open('test.pcl', 'wb') as f:
    cPickle.dump(n, f, -1)

In [12]:
scores_grid

,dataset,RBFNet(tanimoto)_score,RBFNet(tanimoto)_std,XELM(tanimoto)_score,XELM(tanimoto)_std,RBFNet(kulczynski2)_score,RBFNet(kulczynski2)_std,XELM(kulczynski2)_score,XELM(kulczynski2)_std,RandomForestClassifier_score,RandomForestClassifier_std
0,data/5ht2a_ExtFP.libsvm,0.862656,0.013061,0.864155,0.007312,0.860170,0.016741,0.863887,0.007701,0.853076,0.012710
1,data/5ht2c_ExtFP.libsvm,0.871699,0.013833,0.871781,0.015360,0.865028,0.014391,0.872067,0.013354,0.874240,0.011241
2,data/5ht6_ExtFP.libsvm,0.872392,0.028857,0.879077,0.024405,0.871194,0.021631,0.874040,0.024449,0.867162,0.028424
3,data/5ht7_ExtFP.libsvm,0.843625,0.027613,0.849587,0.017068,0.836821,0.018069,0.842112,0.023025,0.807079,0.022839
4,data/M1_ExtFP.libsvm,0.872514,0.020189,0.873793,0.010787,0.868165,0.012254,0.872911,0.009327,0.876354,0.009607
5,data/SERT_ExtFP.libsvm,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
scores_grid.to_csv(r'data/scores_grid.csv', index=False)

a)
    1. outer*n
    2. fix projection, adjust size
    result: std minimization
b) Models then projections then representations

c) github